# Quantum Computation - Week 2

&nbsp;

## Contents

&nbsp;

1. [Working with the IBMQ backends](#ibmq)
    
    1.1 [Experimental properties](#exp)
    
    1.2 [Interactive tools](#ipywid)
    
    1.3 [Quantum half-adder, revisited](#qha)
    
    
2. [Quantum Teleportation protocol](#quantumteleport)

&nbsp;

# 1. Working with the IBMQ backends<a id='ibmq'></a>

&nbsp;

#### Loading the IBMQ account


The `IBMQ` module of Qiskit provides the framework to work with the free-access quantum devices that are currently available.

<div class="alert alert-block alert-info">
    
**Managing your access token**

&nbsp;


Before working with the backends for the first time, a `TOKEN` from an [IBM Quantum Experience](https://quantum-computing.ibm.com/) account is necessary. After creating and logging into an account, the token string can be copied from:

&nbsp;

<p style="text-align: center;"><a href="https://quantum-computing.ibm.com/account">https://quantum-computing.ibm.com/account</a></p>

&nbsp;

This string is used as a personal credential for handling communication with the quantum processors. There are several commands to manage credentials:

- `enable_account('TOKEN')`: Enable account in the current session.
- `save_account('TOKEN')`: Save account to disk for future use.
- `load_account()`: Load account using stored credentials.
- `disable_account()`: Disable account in the current session.
- `stored_account()`: List the account stored to disk.
- `active_account()`: List the account currently in the session.
- `delete_account()`: Delete the saved account from disk.

</div>


#### Module imports

In [ ]:
from qiskit import (QuantumCircuit, ClassicalRegister, QuantumRegister, 
                    IBMQ, BasicAer, execute, transpile)

from qiskit.tools.visualization import plot_histogram, plot_circuit_layout

from qiskit.providers.ibmq import least_busy

from math import pi
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Replace TOKEN with your string

#IBMQ.save_account('TOKEN')

# In subsequent sessios the Token can be loaded from disk

IBMQ.load_account()

&nbsp;

By invoking `get_provider()` one can initiate a provider object to retrieve backend information or selecting a backend:

- `backends()`: Returns all backend objects known to the provider.
- `get_backend('NAME')`: Returns the named backend.

&nbsp;

In [ ]:
provider = IBMQ.get_provider(hub='ibm-q')
provider.backends()

In [ ]:
# Use least_busy() to find out which quantum device is the least busy

physical_devices = provider.backends(simulator=False, operational=True)

backend = least_busy(physical_devices)

# Alternative: get backend directly by name

#backend = provider.get_backend('ibmq_vigo')

print(backend)

## 1.1 Quantum devices - experimental properties<a id='exp'></a>

#### Physical gate set

Although IBM's interface allows the user to program a quantum algorithm using a broad set of single and multi-qubit gates, these are compiled into the two types of quantum operations which can be directly implemented physically. One is a single-qubit unitary operation:

$$U(\theta, \phi, \lambda)$$

The other physically implementable operation is a controlled NOT gate (CNOT, or CX). All quantum algorithms to be performed on current quantum devices need to be decomposed into a sequence of these two types of operations.


#### Qubit map

Besides the restriction regarding the available gates, there are further physical constraints given by the physical architecture of the chip. In fact, CNOT gates can be directly applied only to qubits that are connected.

&nbsp;

<img src="img/qubit_map.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="800 px" align="center">

<p style="text-align: center;">
a) IBM Q5 Tenerife; b) IBM Q20 Tokyo;
</p>

&nbsp;

There are ways to circumvent the chip's architecture limitations (for example, by using SWAP operations to allow for a CNOT gate between otherwise physically uncoupled qubits). 

The study and development of efficient mapping algorithms between logical (software) and physical qubits, for a given quantum program, has only recently started receiving serious attention from the quantum information community. **As the size and complexity of quantum processor chips increase, efficient mapping algorithms will become crucial to successful demonstrations of the applications of quantum computers.**


#### Coherence times

Current quantum computations are “fragile”. A physical qubit does not hold its state indefinitely, but undergoes random bit-flips and loses its phase over time, i.e. undergoes decoherence. Decoherence is the loss of quantum "properties" of a quantum system. Quantum devices have associated decoherence times, which limit the number of quantum operations that can be performed before the results are "drowned" by noise. 

One can distinguish between two measures of decoherence:

1. $T_1$ is the "longitudinal coherence time" (also known as "amplitude damping"), and it measures loss of energy from the system.
2. $T_2$ is the "transverse coherence time" (also known as "phase damping").

<img src="img/coherence_times.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="800 px" align="center">

<div class="alert alert-block alert-info">
    
**How are coherence times determined experimentally?**

&nbsp;

One way to estimate $T_1$ is to initialize a qubit to the ground state $| 0 \rangle$ (for example, apply an $X$ gate to turn it into $| 1 \rangle$, and measure it in the computational basis after a time $t$. 

The probability of the qubit staying in the $| 1 \rangle$ state is expected to follow an exponential decay curve $e^{-t/T_1}$. 

To experimentally determine $T_2$, one can initialise a qubit to the ground state $| 0 \rangle$, apply an Hadamard transform $H$ to change it into $\frac{| 0 \rangle + | 1 \rangle}{\sqrt{2}}$ and wait for a time $t$ before applying another transform $H$ and measuring the qubit on the computational basis.

</div>

&nbsp;

Information on a given `backend`, such as qubit configuration or noise properties, can be retrieved by calling on the instance 
properties.

In [ ]:
# Check backend status
backend.status()

In [ ]:
# Check configuration
config = backend.configuration()

# to_dict() displays configuration in dict form - easier to read
config.to_dict()

In [ ]:
properties = backend.properties()

properties.to_dict()

&nbsp;


## 1.2 Interactive tools <a id='ipywid'></a>

The large amount of information on different backends can better be visualized using the integrated interactive tools.

In [ ]:
# NOTE: if interactive tools don't work,
# you may need to install ipywidgets.
# If so, execute the line below:

# !pip install ipywidgets

# Interactive tools
import qiskit.tools.jupyter 

In [ ]:
%qiskit_backend_overview

After importing the tools, simply invoking a backend instance displays graphical information on the backend.

In [ ]:
backend

&nbsp;


## 1.3 Quantum half-adder, revisited <a id='qha'></a>

&nbsp;


Recall the quantum half-adder circuit from the previous class, which adds the states of $q_0$ and $q_1$, and outputs the *sum* S to $q_1$ and the *carry* C to an additional qubit $q_2$:

&nbsp;

| $q_0$ (input) = A | $q_1$ (input) = B | $q_1$ (output) = S  | $q_2$ (output) = C |
|-----------------|-----------------|-------------------|------------------|
|        0        |         0       |      0            |      0     |
|        1        |         0       |      1            |      0     |
|        0        |         1       |      1            |      0     |
|        1        |         1       |      0            |      1     |


&nbsp;


We can now compare the results of the quantum half adder under ideal conditions (when using the simulator), vs. when executed on a physical quantum processor.

In [ ]:
# Create registers
qr = QuantumRegister(3)
cr = ClassicalRegister(3)

# Quantum Circuit
half_adder = QuantumCircuit(qr, cr)

# We are adding q0 = |1> and q1 = |1>
half_adder.x(qr[0])
half_adder.x(qr[1])

# Perform operation
half_adder.ccx(qr[0], qr[1], qr[2])
half_adder.cx(qr[0], qr[1])

# Barriers make circuits prettier
half_adder.barrier()

# Measure
half_adder.measure(qr, cr)

# Draw
half_adder.draw(output='mpl')

Recall that under noiseless settings and deterministic inputs (since inputs are not in superposition), the output is deterministic: $q_0 = 1$, $q_1 = 0$ (sum) and $q_2 = 1$ (carry). We therefore expect the measured outcome to be `101`.

In [ ]:
sim_backend = BasicAer.get_backend("qasm_simulator")

shots = 1024
job_half_adder_s = execute(half_adder, sim_backend, shots=shots)

result_half_adder_s = job_half_adder_s.result()
counts_half_adder_sim = result_half_adder_s.get_counts(half_adder)

plot_histogram(counts_half_adder_sim)

#### Visualizing circuit decompositions

Before being executed on a physical backend, the circuit is decomposed into unitary and `CNOT` gates. This decomposition can be visualized using the `transpile` function.

In [ ]:
# To visualize the decomposition, we have to specify the backend
# get_backend() to define backend
physical_backend = provider.get_backend('')
                       
# With transpile we can see exactly what is running on the backend
qc_real = transpile(half_adder, backend=backend)

qc_real.draw(output='mpl')

#### Comparing the number of layers

We can compare the number of *layers* between the original and the compiled circuit. 

A *layer* is comprised of all quantum operations that can be executed concurrently, before the quantum device executes the next cycle of operations.

In [ ]:
print('Original circuit length: ', half_adder.depth())
print('Compiled circuit length: ', qc_real.depth())

We can also visually see which qubits are used for the circuit.

In [ ]:
plot_circuit_layout(qc_real, backend)

#### Comparing results

In [ ]:
# Execute the DJ algorithm on physical backend

# Enable job monitoring
%qiskit_job_watcher

# To disable: %qiskit_disable_job_watcher

job_ha_real = execute(half_adder, physical_backend, shots=shots)

In [ ]:
# We can save the job ID to retrieve results even after closing the notebook
job_ha_real.job_id()

# This can be achieved using the retrieve_job('ID') method
#job_ha_real = physical_backend.retrieve_job('ID')

In [ ]:
# We can also check the job status
job_ha_real.status()

In [ ]:
result_ha_real = job_ha_real.result()
counts_ha_real = result_ha_real.get_counts(half_adder)

plot_histogram([counts_ha_real, counts_half_adder_sim], legend=[ 'real device', 'ideal'])

&nbsp;

# 2. Quantum Teleportation protocol<a id='quantumteleport'></a>

&nbsp;

Quantum teleportation is a protocol to transmit quantum states from one location to another, assisted by a previously shared entangled state and a classical communication channel. It was devised by Charles H. Bennett (IBM), Gilles Brassard, Claude Crépeau, Richard Jozsa, Asher Peres, and William K. Wootters in [1993](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.70.1895). It was first demonstrated with photons in [1997](http://www.nature.com/nature/journal/v390/n6660/abs/390575a0.html), and has since been realised in [atoms](http://www.nature.com/nphys/journal/v9/n7/abs/nphys2631.html), [ions](http://www.nature.com/nature/journal/v429/n6993/full/nature02570.html), [electrons](http://science.sciencemag.org/content/345/6196/532) and [superconducting circuits](http://www.nature.com/nature/journal/v500/n7462/full/nature12422.html). The [record](https://www.nature.com/articles/nature23675) distance for quantum teleportation is 1400 km via satellite.


<img src="img/teleportation.png" alt="Note: In order for images to show up in this jupyter notebook you need to select File => Trusted Notebook" width="600 px" align="center">

As illustrated above, the protocol starts out with a shared entangled state between the sender (Alice) and the receiver (Bob):
$$|\psi\rangle_{AB} = \frac{1}{\sqrt{2}}(|0\rangle_A \otimes |0\rangle_B + |1\rangle_A \otimes |1\rangle_B)$$
The first qubit, denoted by subscript $A$, belongs to Alice, and the second qubit, $B$, belongs to Bob.

Alice has a quantum state that she wants to convey to Bob:
$$|\psi\rangle_{C} = \alpha|0\rangle_C + \beta|1\rangle_C$$

At this point, Alice has two quantum states ($C$, the one she wants to teleport, and $A$, one of the entangled pair), and Bob has one quantum state. The total state of the system is given by:
$$|\psi\rangle_{AB} \otimes |\psi\rangle_C = \frac{1}{\sqrt{2}}(|0\rangle_A  |0\rangle_B + |1\rangle_A |1\rangle_B) \otimes (\alpha|0_C\rangle + \beta|1_C\rangle)$$


Alice now measures her two quantum states, $A$ and $C$, in the [Bell basis](https://www.quantiki.org/wiki/bell-basis). This will collapse the three state system into the one of the following four states with equal probability, with the corresponding measurement outcomes:
- **00**: $|\Phi^+\rangle_{AC}\otimes(\alpha|0\rangle_B + \beta|1\rangle_B)$
- **01**: $|\Phi^-\rangle_{AC}\otimes(\alpha|0\rangle_B - \beta|1\rangle_B)$
- **10**: $|\Psi^+\rangle_{AC}\otimes(\alpha|1\rangle_B + \beta|0\rangle_B)$ 
- **11**: $|\Psi^-\rangle_{AC}\otimes(-\alpha|1\rangle_B + \beta|0\rangle_B)$

Alice now sends the results of her measurements to Bob. Using this information, he performs one of the following transformations on his quantum state to transform it to the desired state $\alpha|0\rangle_B + \beta|1\rangle_B$:

- If he receives **00**, he applies $I = \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix}$
- If he receives **01**, he applies $Z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix}$
- If he receives **10**, he applies $X = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix}$ 
- If he receives **11**, he applies $XZ = \begin{pmatrix} 0 & -1 \\ 1 & 0 \end{pmatrix}$ 

Transmission (teleportation) of $|\psi\rangle = \alpha|0\rangle + \beta|1\rangle$ is thus achieved. 

<div class="alert alert-block alert-warning">

**Exercise 2 - Implementing the quantum teleportation protocol**

&nbsp;

Implement the quantum teleportation protocol in the cell below. For this example, we can admit that the quantum state to teleport is $|qc \rangle = \frac{1}{2} |0\rangle + \frac{\sqrt{3}}{2} |1\rangle$.

&nbsp;


The protocol can be broken down into several steps:

- Entangle qubits `qa` and `qb` so that $|qa \; qb \rangle = |00\rangle + |11\rangle$. This can be achieved with a Hadamard and a CNOT gate;


- Prepare the state $|qc\rangle$ as stated above; this can be achieved by a rotation around the y-axis of the qubit, which can be performed by the Ry gate: `ry(pi/3, qc)`;


- To measure Alice's qubits in the [Bell basis](https://www.quantiki.org/wiki/bell-basis), she first has to apply the inverse of the entanglement operation on her two qubits - a `CNOT` followed by a Hadamard (this is already filled below);


- Measure `qa` into bit $b_0$ and `qb` into bit $b_1$; as per the order above, $b_0 b_1 = A \, C$;


- Apply gate `Z` to `qb` if $b_0 = 1$, and apply `X` to `qb` if $b_1 = 1$;


- To verify that the protocol worked correctly, reverse `qb` into $|0\rangle$ by applying the inverse of the initial operation on `qc`: `ry(-pi/3, qb)`


If the protocol was correctly applied, Bob should measure `0` independetly of Alice's results.
</div>


<div class="alert alert-block alert-info">
    
**Quantum operations with classical controls**

Qiskit allows for any quantum operation to be performed conditionally. For that, you need to append `c_if(bit, value)` to the original gate. 

E.g. to apply the Z gate to qubit `qb` if bit `b[0]` == 1: `qc.z(qb).c_if(b[0], 1)`

</div>

In [ ]:
# Qubits A, B and C as separate registers
qc = QuantumRegister(1, 'qc')
qa = QuantumRegister(1, 'qa')
qb = QuantumRegister(1, 'qb')

# Alice needs two bits; Bob needs one
c_alice0 = ClassicalRegister(1)
c_alice1 = ClassicalRegister(1)
c_bob = ClassicalRegister(1)

# Initialize circuit
teleport = QuantumCircuit(qc, qa, qb, c_alice0, c_alice1, c_bob)


# Entangle qa and qb
teleport.h(qa)
teleport.cx(qa, qb)

# Prepare state |qc> on qubit qc
teleport.ry(pi/3, qc)

# Project qc and qa into the Bell basis
teleport.cx(qc, qa)
teleport.h(qc)

# Measure qc and qa into c_alice[0] and c_alice[1]
teleport.measure(qc, c_alice0)
teleport.measure(qa, c_alice1)
teleport.barrier()

# Perform conditional operations on qb
teleport.z(qb).c_if(c_alice0, 1)
teleport.x(qb).c_if(c_alice1, 1)

# Revert qb to |0>
teleport.ry(-pi/3, qb)


# Measure qb to c_bob
teleport.measure(qb, c_bob[0])


# Draw circuit
teleport.draw(output='mpl')